Copyright 2019, Pavel Eftimovski, All rights reserved.

In [8]:
@interact
def check_delivery(Select = X_test_clf.index):
    
    if load_clf.predict(X_test_clf.iloc[Select].values.reshape(1, -1)) == 0:
        isDelayed=False
        print("Delivery with ID:", Select, " will arrive on time.")
        
    elif load_clf.predict(X_test_clf.iloc[Select].values.reshape(1, -1)) == 1:
        isDelayed=True
         
        prediction = load_reg.predict(X_test_reg.iloc[Select].values.reshape(1, -1))
        print()
        print("Delivery with ID:", Select, " will be delayed. \nApproximate time of delay: ",
                                                            round(prediction[0], 3), " days")
        
        for i in range(len(y_test_reg.index)):
            if i == Select:
                df_pred_reg = pd.DataFrame([[y_test_reg.iloc[i]['delay_t'],
                                             prediction]],
                                             columns=['actual', 'predicted'])
                a, b = plt.subplots(1, 1, figsize=(2,4))
                ax = plt.axes()
                ax.set_title("Actual vs. Predicted")
                ax = sns.barplot(data=df_pred_reg,palette="Blues_d", ax=ax)
            else:
                pass
    
    button = widgets.Button(description='Send report',
                            disabled=False,
                            button_style='info',
                            tooltip='True',
                            icon='check')
    
    def generate_report(b):    
        # set report description
        shippingFrom = delivery_desc.iloc[y_ti_reg['index'][Select]]['manu_country']
        shippingTo = delivery_desc.iloc[y_ti_reg['index'][Select]]['country']

        year = str(delivery_desc.iloc[y_ti_reg['index'][Select]]['sch_del_date_yr'])
        month = str(delivery_desc.iloc[y_ti_reg['index'][Select]]['sch_del_date_mn'])
        day = str(delivery_desc.iloc[y_ti_reg['index'][Select]]['sch_del_date_dy'])
        scheduled = month+"/"+day+"/"+year

        if isDelayed:
            send_email(str(Select), shippingFrom, shippingTo, scheduled, True, str(round(prediction[0], 3)))
        else:
            send_email(str(Select), shippingFrom, shippingTo, scheduled, False, "")
            
    display(button)
    button.on_click(generate_report)

interactive(children=(Dropdown(description='Select', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [1]:
import pandas as pd
import seaborn as sns
import pickle
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import matplotlib.pyplot as plt
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
demo_path = '.../USAID_Data/'

# load data
X_test_clf = pd.read_csv(demo_path + 'demo data/X_test_clf.csv')
X_test_reg = pd.read_csv(demo_path + 'demo data/X_test_reg.csv')
y_test_reg = pd.read_csv(demo_path + 'demo data/y_test_reg.csv')
delivery_desc = pd.read_csv(demo_path + 'train_test/test/objects_ts.csv')

# load models
load_clf = pickle.load(open(demo_path+'final model/MLP_Classifier.sav', 'rb'))
load_reg = pickle.load(open(demo_path+'final model/ExtraTrees_Regressor.sav', 'rb'))

y_ti_reg, delivery_desc = [df.rename(columns={'Unnamed: 0': 'index'}) for df in (y_test_reg,
                                                                                 delivery_desc)]

X_test_clf, X_test_reg, y_test_reg = [df.drop(columns=['Unnamed: 0']) for df in (X_test_clf,
                                                                                 X_test_reg,
                                                                                 y_test_reg)]

In [3]:
sender = widgets.Text(placeholder='Enter sender email',description='Sender: ', disabled=False)
sender

Text(value='', description='Sender: ', placeholder='Enter sender email')

In [4]:
receiver = widgets.Text(placeholder='Enter receiver email', description='Receiver: ', disabled=False)
receiver

Text(value='', description='Receiver: ', placeholder='Enter receiver email')

In [5]:
password = widgets.Password(description='Password:')
password

Password(description='Password:')

In [6]:
def send_email(orderNo, shippingFrom, shippingTo, scheduled, isDelayed = False, delayedTime=""):
    message = MIMEMultipart("alternative")
    message["Subject"] = "USAID Supply Chain Report"
    message["From"] = sender.value
    message["To"] = receiver.value

    html = mailBody(orderNo, shippingFrom, shippingTo, scheduled, isDelayed, delayedTime)

    # Turn these into plain/html MIMEText objects
    part = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    message.attach(part)

    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com",465, context=context) as server:
        server.login(sender.value,
                     password.value)

        server.sendmail(sender.value,
                        receiver.value,
                        message.as_string())

In [7]:
def mailBody(orderNo, shippingFrom, shippingTo, scheduled, isDelayed = False, delayedTime=""):
    if isDelayed == True:
        delayedTime = delayedTime
        delayedTime = "Forcasted delay time: <strong>" + delayedTime + " days</strong>"
    else:
        delayedTime=""
        
    html = """\
    <!DOCTYPE html>
    <html>
    <head>
      <title></title>
    </head>
    <body>
      <div class="ii gt" id=":18b">
        <div class="a3s aXjCH msg7957217965308203983" id=":18a">
          <u></u>
          <div style="background:#fff">
            <div style="color:transparent;opacity:0;font-size:0px;border:0;max-height:1px;width:1px;margin:0px;padding:0px;border-width:0px!important;display:none!important;line-height:0px!important"><img border="0" class="CToWUd" height="1" src="https://ci6.googleusercontent.com/proxy/xi-yQXkvbIEymdXtu5Vus_YOvvLB7p_cMqeuGRD9mko6vh3kUBoa4fk7M7dupw2VWACB6IE8-R-yvH7Belg3L_WjaXxXKeEWy2r2gaQOf0L-VV3cIY0VS0LWOqpXO4G68c-YO3whLJHlog6J0baM-e3kI8tsv2Um-CxKq2XuGnCZdOVL15y9mV3sSoo32K4p-HZOq22PKIs0rz0aaohvN1vHruc=s0-d-e1-ft#http://go.sparkpostmail.com/q/2Ure23tJIFvj2WIguYUswQ~~/AAD4-wA~/RgRdeRNOPVcDc3BjQgoAAk6OllvPfmiEUhpsYXphci5zLmdlb3JnaWV2QGdtYWlsLmNvbVgEAAAAAA~~" width="1"></div>
            <div class="m_7957217965308203983mj-container" style="background-color:#fff">
              <div style="margin:0 auto;max-width:960px">
                <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="font-size:0;width:100%">
                  <tbody>
                    <tr>
                      <td style="text-align:center;vertical-align:top;direction:ltr;font-size:0;padding:20px 0;padding-bottom:64px;padding-top:64px">
                        <div class="m_7957217965308203983mj-column-per-100 m_7957217965308203983outlook-group-fix" style="vertical-align:top;display:inline-block;direction:ltr;font-size:13px;text-align:left;width:100%">
                          <table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%">
                            <tbody>
                              <tr>
                                <td align="center" style="word-wrap:break-word;font-size:0;padding:10px 25px;padding-top:0;padding-bottom:0">
                                  <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0">
                                    <tbody>
                                      <tr>
                                        <td style="width:105px">

                                        </td>
                                      </tr>
                                    </tbody>
                                  </table>
                                </td>
                              </tr>
                            </tbody>
                          </table>
                        </div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <div style="margin:0 auto;max-width:960px">
                <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="font-size:0;width:100%">
                  <tbody>
                    <tr>
                      <td style="text-align:center;vertical-align:top;direction:ltr;font-size:0;padding:20px 0;padding-bottom:20px;padding-top:0">
                        <div class="m_7957217965308203983mj-column-px-550 m_7957217965308203983outlook-group-fix" style="vertical-align:top;display:inline-block;direction:ltr;font-size:13px;text-align:left;width:100%">
                          <table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%">
                            <tbody>
                              <tr>
                                <td align="center" style="word-wrap:break-word;font-size:0;padding:10px 25px;padding-top:0;padding-bottom:0">
                                  <div style="color:#191c1e;font-family:Helvetica Neue,Arial;font-size:48px;font-weight:300;line-height:1.16;text-align:center">
                                    USAID Supply Chain Forecast
                                  </div>
                                </td>
                              </tr>
                            </tbody>
                          </table>
                        </div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <div style="margin:0 auto;max-width:960px">
                <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="font-size:0;width:100%">
                  <tbody>
                    <tr>
                      <td style="text-align:center;vertical-align:top;direction:ltr;font-size:0;padding:20px 0;padding-bottom:64px;padding-top:0">
                        <div class="m_7957217965308203983mj-column-px-560 m_7957217965308203983outlook-group-fix" style="vertical-align:top;display:inline-block;direction:ltr;font-size:13px;text-align:left;width:100%">
                          <table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%">
                            <tbody>
                              <tr>
                                <td align="center" style="word-wrap:break-word;font-size:0;padding:10px 25px;padding-top:0;padding-bottom:64px">
                                  <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0">
                                    <tbody>
                                      <tr>
                                        <td style="width:448px">
                                          <img alt="" class="CToWUd a6T" height="auto" src="https://ci5.googleusercontent.com/proxy/_1MEnUtgWOXkzWWqvtrOqCJyTnH71CeQ74b4ZvNvdL-v_aoBtRq0Ek5Zhfudr9mrhHvvkpG4NHdt5uq_JCnvfB-T_AIGP09NDnH4uQJhj9h0POyl-ozo9hFgw4rZhhKoe6rDC1OpWY5j9wk=s0-d-e1-ft#https://s3-eu-west-1.amazonaws.com/revolut-cdn/email/newsletters/card+delivery/Drone.gif" style="border:none;border-radius:0;display:block;font-size:13px;outline:0;text-decoration:none;width:100%;height:auto" tabindex="0" width="448">
                                          <div class="a6S" dir="ltr" style="opacity: 0.01;">
                                            <div aria-label="Download attachment" class="T-I J-J5-Ji aQv T-I-ax7 L3 a5q" data-tooltip-class="a1V" id=":19p" role="button" tabindex="0" title="Download">
                                              <div class="aSK J-J5-Ji aYr"></div>
                                            </div>
                                          </div>
                                        </td>
                                      </tr>
                                    </tbody>
                                  </table>
                                </td>
                              </tr>
                              <tr>
                                <td align="center" style="word-wrap:break-word;font-size:0;padding:0">
                                  <div style="color:#333;font-family:Helvetica Neue,Arial;font-size:16px;font-weight:400;line-height:1.5;text-align:center">
                                    This is a generated report for delivery with ID: 
                                    """ + orderNo + """\
                                    <br/><br/>
                                    Shipped from: 
                                    """ + shippingFrom + """\
                                    <br/>
                                    Destination:  
                                    """ + shippingTo + """\
                                    <br/>
                                    Scheduled delivery date:  
                                    """ + scheduled + """\
                                    <br/> 
                                    """ + delayedTime + """\
                                    <br/>
                                  </div>
                                </td>
                              </tr>
                            </tbody>
                          </table>
                        </div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <div style="margin:0 auto;max-width:960px">
                <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="font-size:0;width:100%">
                  <tbody>
                    <tr>
                      <td style="text-align:center;vertical-align:top;direction:ltr;font-size:0;padding:20px 0;padding-bottom:64px;padding-top:0">
                        <div class="m_7957217965308203983mj-column-px-560 m_7957217965308203983outlook-group-fix" style="vertical-align:top;display:inline-block;direction:ltr;font-size:13px;text-align:left;width:100%">
                          <table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%">
                            <tbody>
                              <tr>
                                <td align="center" style="word-wrap:break-word;font-size:0;padding:10px 25px;padding-top:0;padding-bottom:20px">
                                  <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0">
                                    <tbody>
                                      <tr>
                                        <td style="width:72px"><img alt="" class="CToWUd" height="auto" src="https://ci5.googleusercontent.com/proxy/J5ssTGnZ_RAEq05s9brrdgNdmVxEHXnoteLYsOxGR_zzNNd07LYFizEnj3VxZUeAiaqVCWxqfBQ-fttjDFTSTVeyN7sLVUFmqrIrKup7YYBk6KPkE6fd7sai7S-igMttTk05swlZww=s0-d-e1-ft#https://s3-eu-west-1.amazonaws.com/revolut-cdn/email/newsletters/card+delivery/2.png" style="border:none;border-radius:0;display:block;font-size:13px;outline:0;text-decoration:none;width:100%;height:auto" width="72"></td>
                                      </tr>
                                    </tbody>
                                  </table>
                                </td>
                              </tr>
                            </tbody>
                          </table>
                        </div>
                        iv style="margin:0 auto;max-width:960px;background:#fafafa">
            <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="font-size:0;width:100%;background:#fafafa">
              <tbody>
                <tr>
                  <td style="text-align:center;vertical-align:top;direction:ltr;font-size:0;padding:20px 0;padding-bottom:36px;padding-top:36px">
                    <div class="m_7957217965308203983mj-column-per-100 m_7957217965308203983outlook-group-fix" style="vertical-align:top;display:inline-block;direction:ltr;font-size:13px;text-align:left;width:100%">
                      <table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%">
                        <tbody>
                          <tr>
                            <td>
                              <center style="max-width:320px;margin:0 auto">
                               Gif animation taken from revolut.com
                              </center>
                            </td>
                          </tr>
                        </tbody>
                      </table>
                    </div>
                  </td>
                </tr>
              </tbody>
            </table>
          </div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
          </div></div></div></div></body></html>
    """
    return html